In [1]:
1+1

2

In [2]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}
"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
    prompt = make_prompt(row['context'], row['question'])
    dataset.loc[idx, 'prompt'] = prompt


Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

In [3]:
dataset

,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
...,...,...,...,...,...
107,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,캐릭터가 완료한 퀘스트 중 보상 경험치가 100 이상 200 이하인 퀘스트의 이름과...,"SELECT q.name, q.reward_items FROM quests AS q...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
108,1,CREATE TABLE characters (\n character_id INT ...,"경험이 1000에서 2000 사이인 캐릭터들의 이름, 레벨, 경험치, 아이템 이름 ...","SELECT c.name, c.level, c.experience, i.item_n...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
109,1,CREATE TABLE npcs (\n npc_id INT PRIMARY KEY ...,각 역할별로 npc의 총 수를 알려주세요.,"SELECT role, COUNT(npc_id) FROM npcs GROUP BY ...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
110,1,CREATE TABLE characters (\n character_id INT ...,장비가 장착된 상태인 캐릭터의 플레이어 ID와 아이템 이름을 나열하십시오.,"SELECT T1.player_id, T2.item_name FROM charact...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = 'shangrilar/yi-ko-6b-text2sql'
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

C:\Users\kmr34\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kmr34\.cache\huggingface\hub\models--shangrilar--yi-ko-6b-text2sql. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Plea

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
cannot import name 'get_available_devices' from 'transformers.utils' (C:\Users\kmr34\anaconda3\Lib\site-packages\transformers\utils\__init__.py)

In [ ]:
import time
for batch_size in [1,2,4,8,16,32]:
    start_time = time.time()
    hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
    print(f'{batch_size}: {time.time() - start_time}')

In [ ]:
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)

In [ ]:
for max_num_seqs in [1,2,4,8,16,32]:
    start_time = time.time()
    llm.llm_engine.scheduler_config.max_num_seq = max_num_seqs
    sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
    outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
    print(f'{max_num_seqs}: {time.time() - start_time}')

ERROR: Invalid requirement: 'chromadb=0.5.0'
Hint: = is not a valid operator. Did you mean == ?
